In [ ]:
!pip install python-dotenv
!pip install jiwer
!pip install gradio
!pip install hopsworks

In [44]:
# imports
import pandas as pd
import joblib
import xgboost as xgb # XGboost
from sklearn.model_selection import  train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,explained_variance_score,mean_squared_error
import warnings
import sys
import os
warnings.filterwarnings("ignore")
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [ ]:
import hopsworks
project = hopsworks.login() 
fs = project.get_feature_store() 

In [13]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

In [14]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)
X = X_train.drop(columns=["date"]).fillna(0)
X

,tempmin,feelslikemin,feelslike,dew,humidity,precipcover,windgust,sealevelpressure,visibility
1,-2.507802,-2.375343,-1.814263,-2.780495,-2.183082,-0.685828,-0.650250,1.735510,0.632535
2,-2.345865,-2.171599,-1.462260,-2.261123,-1.972856,-0.685828,-0.096911,1.626981,0.467722
3,-0.969402,-0.867635,-1.075056,-2.185117,-2.596525,-0.685828,-0.918273,1.456435,0.797348
4,-0.839853,-0.758971,-0.934255,-1.843092,-2.295202,-0.685828,-0.667542,1.301393,0.742410
6,-2.216316,-2.456841,-1.872930,-1.893763,-0.984796,-0.685828,1.243206,1.657989,-0.081654
...,...,...,...,...,...,...,...,...,...
360,-1.795280,-1.641863,-1.321459,-1.425061,-0.746541,-0.685828,-0.996087,1.564964,0.687473
361,-1.244695,-1.098545,-1.251058,-1.551737,-1.279112,-0.685828,0.214342,1.704502,0.687473
363,-1.989604,-2.062935,-1.767329,-1.640411,-0.487262,-0.685828,0.283509,2.247147,0.632535
364,-2.167735,-2.144433,-1.814263,-1.577073,-0.234992,-0.685828,0.318093,2.200635,0.742410


In [ ]:
print(y_train)

In [ ]:
gb = GradientBoostingRegressor()
gb.fit(X, y_train)

In [ ]:
gb.get_params()

In [ ]:
test = X_test.drop(columns=["date"]).fillna(0)
y_pred = gb.predict(test)
y_pred=y_pred.round().tolist()
y_pred

In [ ]:
y_test

In [38]:
acc=gb.score(test,y_test)
acc

0.6972055384554663

In [28]:
metrics = classification_report(y_test, y_pred, output_dict=True)

In [ ]:
mr = project.get_model_registry()
model_dir="AIR_Forecast_Model"
if os.path.isdir(model_dir) == False:
   os.mkdir(model_dir)

joblib.dump(gb, model_dir + "/AIR_Forecast_Model.pkl")   

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

AIR_Forecast_model = mr.sklearn.create_model(
    name="AIR_Forecast_Model", 
    metrics={"accuracy" : metrics['accuracy']},
    model_schema=model_schema,
    description="AIR Forecast"
)

AIR_Forecast_model.save(model_dir)

In [51]:
metrics = explained_variance_score(y_test, y_pred, sample_weight=None, multioutput='uniform_average')
metrics

0.6976684466467433

In [ ]:
mr = project.get_model_registry()
model_dir="AIR_Forecast_Model"
if os.path.isdir(model_dir) == False:
   os.mkdir(model_dir)

joblib.dump(gb, model_dir + "/AIR_Forecast_Model.pkl")   

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

AIR_Forecast_model = mr.sklearn.create_model(
    name="AIR_Forecast_Model", 
    metrics={"explained_variance_score" : metrics},
    model_schema=model_schema,
    description="AIR Forecast"
)

AIR_Forecast_model.save(model_dir)

In [ ]:
#metrics = mean_squared_error(y_test, y_pred, sample_weight=None, multioutput='uniform_average')
#metrics
#这个是绝对方差，要使用相关方差